In [13]:
import numpy as np
import pandas as pd
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries 
import config

In [14]:
config.config

<module 'config' from 'E:\\Stock-Crypto_Predictor\\config.py'>

AttributeError: module 'alpha_vantage' has no attribute 'requests'